In [31]:
from fredapi import Fred

api_key_fred = "" 
fred = Fred(api_key=api_key_fred)

In [32]:
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import plotly.graph_objects as go
import datetime
import pandas as pd
import csv
import os
import time
from dateutil import tz
import glob
import quandl as q
import re
import plotly.graph_objects as go
import numpy as np

quandl_api_key = os.environ['quandl_api_key']

# # # start - read in BTC data # # #
datasource_btcusd = "BITFINEX/BTCUSD.csv"
btcusd_data = pd.read_csv("coindata/{}".format(
    datasource_btcusd.replace("/", " ")), index_col=0)
btcusd_data.index = pd.to_datetime(btcusd_data.index)

most_recent_stored_btcusd_date = btcusd_data.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")
todays_date = datetime.date.today() - timedelta(days=1)
todays_date = todays_date.strftime("%Y-%m-%d")


if most_recent_stored_btcusd_date != todays_date:
    # start - get data from quandl api
    data = q.get(datasource_btcusd.split(".")[0],   start_date=most_recent_stored_btcusd_date,
                    end_date='{}'.format(todays_date),
                    api_key=quandl_api_key)
    data.info()
    data["First"] = data.Last.shift(1)
    data.dropna()
    btcusd_data = pd.concat([btcusd_data, data])
    btcusd_data = btcusd_data.sort_index()
    # store current df with up-to-date values
    btcusd_data.to_csv('coindata/{}'.format(
        datasource_btcusd.replace("/", " ")), index=True)
        
# # # end - read in BTC data # # #

In [33]:
fed_assets_quandl_key = "FED/RESPPA_N_WW"
fed_assets_data = q.get(fed_assets_quandl_key, 
            api_key=quandl_api_key
            )
fed_assets_data = fed_assets_data.dropna()
fed_assets_data.info()
fed_assets_data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1039 entries, 2002-12-18 to 2022-11-09
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Value   1039 non-null   float64
dtypes: float64(1)
memory usage: 16.2 KB


,Value
Date,
2002-12-18,720761.0
2002-12-25,733136.0
2003-01-01,732202.0
2003-01-08,724902.0
2003-01-15,721325.0
...,...
2022-10-12,8758969.0
2022-10-19,8743922.0
2022-10-26,8723090.0


In [34]:
fed_assets_data = fed_assets_data.sort_index()
fed_assets_data

,Value
Date,
2002-12-18,720761.0
2002-12-25,733136.0
2003-01-01,732202.0
2003-01-08,724902.0
2003-01-15,721325.0
...,...
2022-10-12,8758969.0
2022-10-19,8743922.0
2022-10-26,8723090.0


In [35]:
# store current df with up-to-date values
fed_assets_data.to_csv('coindata/{}'.format(
    fed_assets_quandl_key.replace("/", " ")), index=True)

In [36]:
# merge fed and btc
btcusd_data_and_fed = pd.merge(btcusd_data, fed_assets_data, left_index=True, right_index=True, how='left')
btcusd_data_and_fed

,High,Low,Mid,Last,Bid,Ask,Volume,First,Value
Date,,,,,,,,,
2014-04-15,513.9000,452.00,504.23500,505.0000,503.5000,504.97,21013.584774,NaN,NaN
2014-04-16,547.0000,495.00,537.50000,538.0000,537.0000,538.00,29633.358705,505.00,4283967.0
2014-04-17,538.5000,486.10,507.02000,508.0000,506.0400,508.00,20709.783819,538.00,NaN
2014-04-18,509.0000,474.25,483.77000,482.7500,482.7500,484.79,10458.045243,508.00,NaN
2014-04-19,513.9899,473.83,505.01065,507.4999,502.5313,507.49,8963.618369,482.75,NaN
...,...,...,...,...,...,...,...,...,...
2022-11-11,17671.0000,16355.00,16991.50000,16991.0000,16991.0000,16992.00,8005.671146,17561.00,NaN
2022-11-12,17106.0000,16635.00,16799.50000,16800.0000,16799.0000,16800.00,1245.280402,16991.00,NaN
2022-11-13,16956.0000,16255.00,16343.50000,16343.0000,16343.0000,16344.00,2534.653007,16800.00,NaN


In [37]:
btcusd_data_and_fed["Value"] = btcusd_data_and_fed["Value"].ffill()
btcusd_data_and_fed = btcusd_data_and_fed.dropna()

In [38]:
btcusd_data_and_fed["BTC_per_FedAssets"] = (btcusd_data_and_fed["Last"] / btcusd_data_and_fed["Value"]) *1000
btcusd_data_and_fed["350_movingaverage_per_FedAssets"] = pd.Series.rolling(btcusd_data_and_fed["BTC_per_FedAssets"], window=350, min_periods=1).mean()
btcusd_data_and_fed["111_movingaverage_per_FedAssets"] = pd.Series.rolling(btcusd_data_and_fed["BTC_per_FedAssets"], window=111, min_periods=1).mean()

C:\Users\gebel\AppData\Local\Temp\ipykernel_23012\2281456468.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\gebel\AppData\Local\Temp\ipykernel_23012\2281456468.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\gebel\AppData\Local\Temp\ipykernel_23012\2281456468.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

# GET S&P DATA

In [100]:
# Billions of Dollars 
data_SP500 = fred.get_series('SP500')
data_SP500 = data_SP500 * 1000

data_SP500.dropna()
data_SP500 = data_SP500.sort_index()
# store current df with up-to-date values
data_SP500.to_csv('coindata/data_SP500.csv', index=True)

print(data_SP500.head(1))
print(data_SP500.tail(1))

2012-11-15    1353330.0
dtype: float64
2022-11-14    3957250.0
dtype: float64


# Assets: Total Assets: Total Assets: Wednesday Level (RESPPANWW)

In [39]:
start_date='2014-04-21'

In [40]:
#Assets: Total Assets: Total Assets: Wednesday Level (RESPPANWW) Millions of Dollars
data_WALCL = fred.get_series('WALCL')

data_WALCL.dropna()
data_WALCL = data_WALCL.sort_index()
# store current df with up-to-date values
data_WALCL.to_csv('coindata/data_WALCL.csv', index=True)

print(data_WALCL.head(1))
print(data_WALCL.tail(1))

2002-12-18    719542.0
dtype: float64
2022-11-09    8678886.0
dtype: float64


# Overnight Reverse Repurchase Agreements Treasury Securities Sold by the Federal Reserve in the Temporary Open Market Operations (RRPONTSYD)

In [41]:
data_RRPONTSYD = fred.get_series('RRPONTSYD')
data_RRPONTSYD

2003-02-07       2.500
2003-02-10         NaN
2003-02-11         NaN
2003-02-12         NaN
2003-02-13         NaN
                ...   
2022-11-08    2232.555
2022-11-09    2237.812
2022-11-10    2200.586
2022-11-11         NaN
2022-11-14    2165.018
Length: 5157, dtype: float64

In [42]:
#Assets: Total Assets: Total Assets: Wednesday Level (RESPPANWW)
# Billions of U.S. Dollars
# data_FRED_RRPONTSYD.values = data_FRED_RRPONTSYD.values * 1000 
data_RRPONTSYD = data_RRPONTSYD * 1000 
data_RRPONTSYD.dropna()
data_RRPONTSYD = data_RRPONTSYD.sort_index()
# store current df with up-to-date values
data_RRPONTSYD.to_csv('coindata/data_RRPONTSYD.csv', index=True)

print(data_RRPONTSYD.head(1))
print(data_RRPONTSYD.tail(1))

2003-02-07    2500.0
dtype: float64
2022-11-14    2165018.0
dtype: float64


# Deposits with Federal Reserve Banks, other than Reserve Balances: U.S. Treasury, General Account (WTREGEN)

In [43]:
# Billions of Dollars 
data_FRED_WTREGEN = fred.get_series('WTREGEN')
data_FRED_WTREGEN = data_FRED_WTREGEN * 1000

data_FRED_WTREGEN.dropna()
data_FRED_WTREGEN = data_FRED_WTREGEN.sort_index()
# store current df with up-to-date values
data_FRED_WTREGEN.to_csv('coindata/data_FRED_WTREGEN.csv', index=True)

print(data_FRED_WTREGEN.head(1))
print(data_FRED_WTREGEN.tail(1))

1986-01-08    4140.0
dtype: float64
2022-11-09    527479.0
dtype: float64


# net liquidity vs s&p500 weekly - Diagram

In [44]:
datasource_fred_total_assets = "data_WALCL.csv"
fred_total_assets = pd.read_csv("coindata/{}".format(
    datasource_fred_total_assets.replace("/", " ")), index_col=0)
fred_total_assets.index = pd.to_datetime(fred_total_assets.index)

most_recent_stored_fred_rrpontsyd_date = fred_total_assets.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")
todays_date = datetime.date.today() - timedelta(days=1)
todays_date = todays_date.strftime("%Y-%m-%d")
fred_total_assets

,0
2002-12-18,719542.0
2002-12-25,732059.0
2003-01-01,730994.0
2003-01-08,723762.0
2003-01-15,720074.0
...,...
2022-10-12,8758969.0
2022-10-19,8743922.0
2022-10-26,8723090.0
2022-11-02,8676870.0


In [45]:
datasource_FRED_WTREGEN = "data_FRED_WTREGEN.csv"
FRED_WTREGEN_data = pd.read_csv("coindata/{}".format(
    datasource_FRED_WTREGEN.replace("/", " ")), index_col=0)
FRED_WTREGEN_data.index = pd.to_datetime(FRED_WTREGEN_data.index)

most_recent_stored_FRED_WTREGEN_date = FRED_WTREGEN_data.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")
FRED_WTREGEN_data

,0
1986-01-08,4140.0
1986-01-15,5032.0
1986-01-22,13856.0
1986-01-29,17487.0
1986-02-05,12475.0
...,...
2022-10-12,608302.0
2022-10-19,607199.0
2022-10-26,634548.0
2022-11-02,598544.0


In [46]:
datasource_FRED_RRPONTSYD = "data_RRPONTSYD.csv"
FRED_RRPONTSYD_data = pd.read_csv("coindata/{}".format(
    datasource_FRED_RRPONTSYD.replace("/", " ")), index_col=0)
FRED_RRPONTSYD_data.index = pd.to_datetime(FRED_RRPONTSYD_data.index)

most_recent_stored_FRED_RRPONTSYD_date = FRED_RRPONTSYD_data.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")
FRED_RRPONTSYD_data

,0
2003-02-07,2500.0
2003-02-10,NaN
2003-02-11,NaN
2003-02-12,NaN
2003-02-13,NaN
...,...
2022-11-08,2232555.0
2022-11-09,2237812.0
2022-11-10,2200586.0
2022-11-11,NaN


In [56]:
type( FRED_RRPONTSYD_data.iloc[0].name )

pandas._libs.tslibs.timestamps.Timestamp

# filter for datetime to allign all fred data sources!

In [75]:
merged_FRED_RRPONTSYD_data = pd.merge(FRED_WTREGEN_data,FRED_RRPONTSYD_data, how='left', left_index=True, right_index=True)
# merged.isnull().sum()
del merged_FRED_RRPONTSYD_data["0_x"]
merged_FRED_RRPONTSYD_data.columns=["0"]

In [88]:
netLiquidity = fred_total_assets-merged_FRED_RRPONTSYD_data-FRED_WTREGEN_data

In [80]:
print(len(merged_FRED_RRPONTSYD_data))
print(len(fred_total_assets))
print(len(FRED_WTREGEN_data))

639
639
639


In [81]:
# df['dt'] = pd.to_datetime(df['dt'])
FRED_RRPONTSYD_data = FRED_RRPONTSYD_data[(FRED_RRPONTSYD_data.index > '2010-08-11 00:00:00')]
fred_total_assets = fred_total_assets[(fred_total_assets.index > '2010-08-11 00:00:00')]
FRED_WTREGEN_data = FRED_WTREGEN_data[(FRED_WTREGEN_data.index > '2010-08-11 00:00:00')]

fred_total_assets.head()
fred_total_assets.columns = ['Value_fred_total_assets']
len(fred_total_assets)

FRED_WTREGEN_data.head()
FRED_WTREGEN_data.columns = ['Value_FRED_WTREGEN_data']
len(FRED_WTREGEN_data)

FRED_RRPONTSYD_data.head()
FRED_RRPONTSYD_data.columns = ['Value_FRED_RRPONTSYD_data']
len(FRED_RRPONTSYD_data)

# bring dfs together and forwardfill all!
frames = [FRED_RRPONTSYD_data, FRED_WTREGEN_data, fred_total_assets]
result = pd.concat(frames)

In [82]:
# result["netLIQ"] = result['Value_FRED_RRPONTSYD_data'] - result['Value_FRED_WTREGEN_data'] - result['Value_fred_total_assets'] 

In [83]:
# result["netLIQ"] = result['Value_FRED_RRPONTSYD_data'] - result['Value_FRED_WTREGEN_data'] - result['Value_fred_total_assets'] 

In [109]:
data_SP500_2weeksback = data_SP500.shift(-2,"W")
data_SP500_2weeksback

2012-11-04    1353330.0
2012-11-04    1359880.0
2012-11-11    1386890.0
2012-11-11    1387810.0
2012-11-11    1391030.0
                ...    
2022-10-30    3828110.0
2022-10-30    3748570.0
2022-10-30    3956370.0
2022-10-30    3992930.0
2022-11-06    3957250.0
Length: 2608, dtype: float64

In [114]:
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(
    go.Scatter(
        x=btcusd_data_and_fed.index,
        y=btcusd_data_and_fed['Last'],
        name="BTC",
        mode='lines',
        marker=dict(
            # size=16,
            color="red",  # set color equal to a variable
            # colorscale='Viridis', # one of plotly colorscales
            # showscale=True
        ), 
    ), 
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(
        x=data_SP500_2weeksback.index,
        y=data_SP500_2weeksback.values,
        name="SP500 - 2 weeks ago",
        mode='lines',
        marker=dict(
            # size=16,
            color="blue",  # set color equal to a variable
            # colorscale='Viridis', # one of plotly colorscales
            # showscale=True
        ), 
    ), 
    secondary_y=True,
)



fig.add_trace(go.Scatter(x=fred_total_assets.index, y=fred_total_assets["0"],
                    mode='lines',
                    name='fred_total_assets',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="black"
                        ),
                    )
                )

fig.update_layout(yaxis=dict(domain=[0, 0.7]) )

fig.add_trace(go.Scatter(x=netLiquidity.index, y=netLiquidity["0"], stackgroup='one', #fill='tonexty',
                    mode='lines',
                    name='netLiquidity',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="green"
                        )
                    )
                )

fig.add_trace(go.Scatter(x=merged_FRED_RRPONTSYD_data.index, y=merged_FRED_RRPONTSYD_data["0"], stackgroup='one', #fill='tonexty',
                    mode='lines',
                    name='merged_FRED_RRPONTSYD_data',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="yellow"
                        )
                    )
                )

fig.add_trace(go.Scatter(x=FRED_WTREGEN_data.index, y=FRED_WTREGEN_data["0"], stackgroup='one', #fill='tonexty',
                    mode='lines',
                    name='FRED_WTREGEN_data',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="red"
                        ),
                    )
                )

                
# btcusd_data["111_movingaverage"]
# fig.update_yaxes(type="log") #, range=[0,5]

fig.update_layout(
    # title="Plot Title",
    autosize=False,
    width=int(1400/1.3),
    height=int(800/1.3),
)
fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)
fig

In [85]:
aaaaasdasd

NameError: name 'aaaaasdasd' is not defined

In [ ]:
import plotly.express as px
df = px.data.gapminder()
fig = px.area(df, x="year", y="pop", color="continent", line_group="country")
fig.show()

In [ ]:
#Assets: Total Assets: Total Assets: Wednesday Level (RESPPANWW)
fed_assets_quandl_key = "FED/RESPPA_N_WW"

fed_assets_data = pd.read_csv("coindata/{}".format(
    fed_assets_quandl_key.replace("/", " ")), index_col=0)
fed_assets_data.index = pd.to_datetime(fed_assets_data.index)

In [ ]:
fed_assets_data.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")

In [ ]:
from datetime import datetime, timedelta
import datetime
most_recent_stored_fed_assets_date = fed_assets_data.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")

todays_date = datetime.date.today() - timedelta(days=7)
todays_date = todays_date.strftime("%Y-%m-%d")


In [ ]:
most_recent_stored_fed_assets_date

In [ ]:
todays_date

In [ ]:
fig = go.Figure(
    data=go.Scatter(
        x=btcusd_data_and_fed.index,
        y=btcusd_data_and_fed['BTC_per_FedAssets'],
        name="BTC/FED",
        mode='lines',
        marker=dict(
            # size=16,
            color="red",  # set color equal to a variable
            # colorscale='Viridis', # one of plotly colorscales
            # showscale=True
        )
)
)

fig.add_trace(go.Scatter(x=btcusd_data_and_fed.index, y=btcusd_data_and_fed["350_movingaverage_per_FedAssets"],
                    mode='lines',
                    name='350_movingaverage_per_FedAssets',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="gold"
                        ),
                    )
                )

fig.add_trace(go.Scatter(x=btcusd_data_and_fed.index, y=btcusd_data_and_fed["350_movingaverage_per_FedAssets"]*2,
                    mode='lines',
                    name='2*350_movingaverage_per_FedAssets',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="red"
                        ),
                    )
                )

fig.add_trace(go.Scatter(x=btcusd_data.index, y=btcusd_data_and_fed["111_movingaverage_per_FedAssets"],
                    mode='lines',
                    name='111_movingaverage_per_FedAssets',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="green"
                        )
                    )
                )

# btcusd_data["111_movingaverage"]
fig.update_yaxes(type="log") #, range=[0,5]

fig.update_layout(
    # title="Plot Title",
    autosize=False,
    width=int(1400/1.3),
    height=int(800/1.3),
)

fig

In [ ]:
btcusd_data = btcusd_data.dropna()
btcusd_data["350_movingaverage"] = pd.Series.rolling(btcusd_data["Last"], window=350, min_periods=1).mean()
btcusd_data["111_movingaverage"] = pd.Series.rolling(btcusd_data["Last"], window=111, min_periods=1).mean()
btcusd_data

In [ ]:
fig = go.Figure(
    data=go.Scatter(
        x=btcusd_data.index,
        y=btcusd_data['Last'],
        mode='lines',
        marker=dict(
            # size=16,
            color="black",  # set color equal to a variable
            # colorscale='Viridis', # one of plotly colorscales
            # showscale=True
        )
    )
)

fig.add_trace(go.Scatter(x=btcusd_data.index, y=btcusd_data["350_movingaverage"]*2,
                    mode='lines',
                    name='350_movingaverage',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="red"
                        ),
                    )
                )

fig.add_trace(go.Scatter(x=btcusd_data.index, y=btcusd_data["111_movingaverage"],
                    mode='lines',
                    name='111_movingaverage',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="green"
                        )
                    )
                )

# btcusd_data["111_movingaverage"]
fig.update_yaxes(type="log") #, range=[0,5]

fig.update_layout(
    # title="Plot Title",
    autosize=False,
    width=int(1400/1.3),
    height=int(800/1.3),
)

fig